In [1]:
import polars as pl
import numpy as np

In [2]:
#Question 1a:
economics_long = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv"
)
economics_wide = (
    economics_long
    .pivot(
        values="value",
        index="date",
        columns="variable",
        aggregate_function="first"
    )
)

economics_wide


/tmp/ipython-input-1719110194.py:7: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


date,pce,pop,psavert,uempmed,unemploy
str,f64,f64,f64,f64,f64
"""1967-07-01""",506.7,198712.0,12.6,4.5,2944.0
"""1967-08-01""",509.8,198911.0,12.6,4.7,2945.0
"""1967-09-01""",515.6,199113.0,11.9,4.6,2958.0
"""1967-10-01""",512.2,199311.0,12.9,4.9,3143.0
"""1967-11-01""",517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
"""2014-12-01""",12062.0,319746.157,7.6,12.9,8717.0
"""2015-01-01""",12046.0,319928.646,7.7,13.2,8903.0
"""2015-02-01""",12082.4,320074.511,7.9,12.9,8610.0


In [3]:
# Question 1b:
economics_wide = economics_wide.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d", strict=False)
)
median_unemploy = (
    economics_wide
    .filter(pl.col("date") >= pl.date(2010, 1, 1))
    .select(pl.col("unemploy").median().alias("median_unemploy"))
    .item()
)

print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")


The median unemployment for 2010-01-01 and beyond is: 12471.0


In [7]:
# Question 2a:
air_passengers = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt",
    separator="\t"
)

print("Original data shape:", air_passengers.shape)
print("\nFirst few rows:")
print(air_passengers.head())

year_col = "year" if "year" in air_passengers.columns else "Year"

air_passengers_long = (
    air_passengers.unpivot(
        index=[year_col],
        on=[c for c in air_passengers.columns if c != year_col],
        variable_name="month",
        value_name="passengers"
    )
    .rename({year_col: "year"})
    .with_columns([
        pl.col("year").cast(pl.Int64),
        pl.col("month").cast(pl.Utf8),
        pl.col("passengers").cast(pl.Int64),
    ])
)

air_passengers_long



Original data shape: (12, 13)

First few rows:
shape: (5, 13)
┌──────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┐
│ Year ┆ Jan ┆ Feb ┆ Mar ┆ … ┆ Sep ┆ Oct ┆ Nov ┆ Dec │
│ ---  ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ i64  ┆ i64 ┆ i64 ┆ i64 ┆   ┆ i64 ┆ i64 ┆ i64 ┆ i64 │
╞══════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ 1949 ┆ 112 ┆ 118 ┆ 132 ┆ … ┆ 136 ┆ 119 ┆ 104 ┆ 118 │
│ 1950 ┆ 115 ┆ 126 ┆ 141 ┆ … ┆ 158 ┆ 133 ┆ 114 ┆ 140 │
│ 1951 ┆ 145 ┆ 150 ┆ 178 ┆ … ┆ 184 ┆ 162 ┆ 146 ┆ 166 │
│ 1952 ┆ 171 ┆ 180 ┆ 193 ┆ … ┆ 209 ┆ 191 ┆ 172 ┆ 194 │
│ 1953 ┆ 196 ┆ 196 ┆ 236 ┆ … ┆ 237 ┆ 211 ┆ 180 ┆ 201 │
└──────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴─────┘


year,month,passengers
i64,str,i64
1949,"""Jan""",112
1950,"""Jan""",115
1951,"""Jan""",145
1952,"""Jan""",171
1953,"""Jan""",196
…,…,…
1956,"""Dec""",306
1957,"""Dec""",336
1958,"""Dec""",337


In [8]:
# Question 2b:
std_passengers = (
    air_passengers_long
    .filter((pl.col("year") >= 1955) & (pl.col("year") <= 1960))
    .select(pl.col("passengers").std().alias("std_passengers"))
    .item()
)

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")


The standard deviation of airline passengers between 1955 and 1960 is: 86.44


In [9]:
# Question 3a:
table2 = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv"
)

print("Original data:")
print(table2.head())

table2_wide = (
    table2
    .pivot(
        values="count",
        index=["country", "year"],
        columns="type",
        aggregate_function="first"
    )
    .with_columns([
        pl.col("year").cast(pl.Int64),
        pl.col("cases").cast(pl.Float64),
        pl.col("population").cast(pl.Float64)
    ])
)

table2_wide


Original data:
shape: (5, 5)
┌──────────┬─────────────┬──────┬────────────┬──────────┐
│ rownames ┆ country     ┆ year ┆ type       ┆ count    │
│ ---      ┆ ---         ┆ ---  ┆ ---        ┆ ---      │
│ i64      ┆ str         ┆ i64  ┆ str        ┆ i64      │
╞══════════╪═════════════╪══════╪════════════╪══════════╡
│ 1        ┆ Afghanistan ┆ 1999 ┆ cases      ┆ 745      │
│ 2        ┆ Afghanistan ┆ 1999 ┆ population ┆ 19987071 │
│ 3        ┆ Afghanistan ┆ 2000 ┆ cases      ┆ 2666     │
│ 4        ┆ Afghanistan ┆ 2000 ┆ population ┆ 20595360 │
│ 5        ┆ Brazil      ┆ 1999 ┆ cases      ┆ 37737    │
└──────────┴─────────────┴──────┴────────────┴──────────┘


/tmp/ipython-input-263062455.py:11: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


country,year,cases,population
str,i64,f64,f64
"""Afghanistan""",1999,745.0,1.9987071e7
"""Afghanistan""",2000,2666.0,2.059536e7
"""Brazil""",1999,37737.0,1.72006362e8
"""Brazil""",2000,80488.0,1.74504898e8
"""China""",1999,212258.0,1.2729e9
"""China""",2000,213766.0,1.2804e9


In [13]:
# Question 3b:
table2_with_ratio = table2_wide.with_columns(
    (pl.col("cases") / pl.col("population")).alias("ratio")
)

year_1999 = table2_with_ratio.filter(pl.col("year") == 1999)


highest_ratio_country = (
    year_1999
    .sort("ratio", descending=True)
    .select("country")
    .head(1)
    .item()
)

print("Data for 1999 with ratios:")
print(year_1999)
print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬──────────┬──────────────┬──────────┐
│ country     ┆ year ┆ cases    ┆ population   ┆ ratio    │
│ ---         ┆ ---  ┆ ---      ┆ ---          ┆ ---      │
│ str         ┆ i64  ┆ f64      ┆ f64          ┆ f64      │
╞═════════════╪══════╪══════════╪══════════════╪══════════╡
│ Afghanistan ┆ 1999 ┆ 745.0    ┆ 1.9987071e7  ┆ 0.000037 │
│ Brazil      ┆ 1999 ┆ 37737.0  ┆ 1.72006362e8 ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258.0 ┆ 1.2729e9     ┆ 0.000167 │
└─────────────┴──────┴──────────┴──────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil
